# Rabljeni motocikli

V nadaljevanju bom analizirala motocikle, ki se prodajajo na http://www.mobile.de. Izbrala sem le oglase rabljenih motociklov, mlajših od 10 let, ter s prostornino večjo od 500ccm.

In [1]:
import pandas as pd

In [14]:
tabela = pd.read_csv('motocikli.csv', index_col = 'id', encoding = 'latin1')
pd.options.display.max_rows = 8

### Razlaga zajetih podatkov

- letnik: leto prve registracije motocikla, za katero bom privzela, da se ujema z letom izdelave
- znamka: ime izdelovalca motocikla
- tip: motocikli so na grobo razdeljeni v skupine glede na obliko in namen
- prostornina: delovna prostornina pogonskega agregata
- moc: moč pogonskega agregata izražena v kW
- km: število prevoženih kilometrov
- cena: cena motocikla, določena s strani prodajalca v evrih

In [17]:
motocikli = tabela.dropna(how='any')
motocikli

,letnik,znamka,tip,prostornina,moc,km,cena
id,,,,,,,
234776213,2006,Hyosung,Chopper And Cruiser,650,55,17000,2800
234776213,2006,Hyosung,Chopper And Cruiser,647,59,3945,2999
229082730,2006,Yamaha,Chopper And Cruiser,649,29,46987,3499
235377812,2006,Harley-Davidson,Chopper And Cruiser,1131,86,3900,21490
...,...,...,...,...,...,...,...
233263400,2016,Harley-Davidson,Tourer,1802,72,650,38900
233584627,2016,Harley-Davidson,Tourer,1801,72,800,39999
233824453,2016,Harley-Davidson,Tourer,1800,72,581,40990
233824453,2016,Harley-Davidson,Tourer,1690,64,900,59900


Poglejmo najprej nekaj povprečnih rezultatov vseh oglasov.

In [18]:
motocikli['znamka'].value_counts()

BMW                3249
Harley-Davidson    2557
Kawasaki           2158
Honda              2127
                   ... 
SMC                   1
Norton                1
Gasgas                1
TM                    1
Name: znamka, dtype: int64

In [19]:
motocikli['letnik'].mean()

2011.354626362631

In [20]:
motocikli['km'].mean()

15809.517931433988

In [21]:
motocikli['cena'].mean()

9724.170519774607

In [22]:
motocikli['tip'].value_counts()

Naked Bike                      5127
Chopper And Cruiser             3690
Sports And Super Sports Bike    3488
Enduro And Touring Enduro       2952
Sport Touring Motorcycle        1935
Tourer                          1173
Super Moto                       624
Name: tip, dtype: int64

In [23]:
motocikli['moc'].mean()

80.23171309705619

Ker nam povprečne vrednosti povedo le zanimive in ne koristnih podatkov, si podatke poglejmo bolj podrobno.

Recimo, da kupujemo motocikel in bi radi svoj denar čim bolje izkoristili. Poglejmo si kako cena narašča oziroma pada glede na izbrane parametre.

Kateri tip motocikla je najdražji?

In [68]:
tipi = motocikli['tip'].unique()
vsebina = {'tip':tipi}
tipi_glede_na_ceno = pd.DataFrame(vsebina)
povprecna_cena_za_tip = motocikli[motocikli['tip']==tipi_glede_na_ceno[:1]].mean()
tipi_glede_na_ceno['povprecna cena'] = motocikli.apply(povprecna_cena_za_tip, axis=1)
tipi_glede_na_ceno

TypeError: unorderable types: int() > str()